In [1]:
import csv
import re
import math
from collections import Counter

with open("mega2.csv") as f:
 
    r = csv.reader(f, delimiter='\t')
    rgx = re.compile(r'\b[a-zA-Z]+\b')
    docs = [ (' '.join(re.findall(rgx, x[2])).lower(), ' '.join(re.findall(rgx, x[3])).lower())  \
            for i,x in enumerate(r)]
 
#print(docs[0][0],docs[0][1])
#print "\n"
#print(docs[11][0],docs[11][1])


# print "\n"
# print(docs[2][0],docs[2][1])
# print "\n"
# print(docs[3][0],docs[3][1])
# print "\n"
# print(docs[4][0],docs[4][1])
 
items_t = [ d[0] for d in docs ] # item titlescd    
items_d = [ d[1] for d in docs ] # item descriptions
items_i = range(0, len(items_t)) # item id


#index
def create_inverted_index(corpus):
   idx={}
   for i, document in enumerate(corpus):
       for word in document.split():
           if word in idx:
              if i in idx[word]:
                idx[word][i] += 1
              else:
                  idx[word][i] = 1
           else:
               idx[word] = {i:1}
   return idx

#idx = create_inverted_index(items_d)
#print(idx)




def idf(term, idx, n):
    return math.log(float(n) / (1 + len(idx[term])))    


def print_results(results,n, head=True):

   if head:    
       print('\nTop %d from recall set of %d items:' % (n,len(results)))
       for r in results[:n]:
           print('\t%0.2f - %s'%(r[0],items_t[r[1]]))

   else:
       print('\nBottom %d from recall set of %d items:' % (n,len(results)))
       for r in results[-n:]:
           print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
 
#BM25

def get_results_bm25(qry, corpus, k1=1.5, b=0.75):
    idx = create_inverted_index(corpus)
    # 1.Assign (integer) n to be the number of documents in the corpus
    ## HIDE
    n = len(corpus)
    # 2.Assign (list) d with elements corresponding to the number of terms in each document in the corpus
    ## HIDE
    d = [len(x.split()) for x in corpus]
    # 3.Assign (float) d_avg as the average document length of the documents in the corpus
    ## HIDE
    d_avg = float(sum(d)) / len(d)                
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                # 4.Assign (float) f equal to the number of times the term appears in doc
                ## HIDE
                f = float(idx[term][doc])
                # 5.Assign (float) s the BM25 score for this (term, document) pair
                # HIDE
                s = i * (( f * (k1 + 1) ) / (f + k1 * (1 - b + (b * (float(d[doc]) / d_avg)))))
                score[doc] += s
                
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results


# #TF IDF
# def get_results_tfidf(qry, idx, n):
#   score = Counter()
  
#   for term in qry.split():
#       # << IMPLEMENT TF-IDF SCORING >> CODE HERE
 
#     if term in idx:
#         i = idf(term, idx, n)
#         #print i
#         for doc in idx[term]:
#             #print "doc", doc
#             #print "idx[term][doc]", idx[term][doc]
#             score[doc] += idx[term][doc] + i
#             #print score
 
#   results=[]
#   for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
#       if x[1] > 0:
#           results.append([x[1],x[0]])
 
#   sorted_results = sorted(results, key=lambda t: t[0] * -1 )
#   return sorted_results
 

#results = get_results_bm25('breast cancer', items_d)

#print results
#print_results(results,5)

#///////////


In [2]:
from ipywidgets import *
from IPython.display import display

#Input
search = Button(button_style = 'danger', description = "Search")
userinput = Text(placeholder="Enter your search")
maxresults = IntSlider(description = "Max results",max=30, value = 10)
#Layout Organization
ButtonGroup =[search, maxresults]
#HTML
HTMLResults = HTML()
PageTitle = HTML()
#Parameters
PageTitle.value += '<p style="text-align: center;"><span style="color: #339966;"><strong>Medical research paper results for</strong></span></p>'

display(userinput, ButtonGroup, widgets.HBox(ButtonGroup))

def searchclick(b):
    #global result
    inputstr = userinput.value
    inputstr.lower()
    #print(userinput.value)
    #result = get_results_bm25(inputstr, idx, len(items_t))
    result = get_results_bm25(inputstr, items_d)
    print(result)
    HTMLResults.value = ''
    HTMLResults.value += '<p style="text-align: center;"><span style="color: #339966;"><strong>%s</strong></span></p>' % (userinput.value)
    for r in result[:maxresults.value]:
        HTMLResults.value += '<p style="text-align: left;"><span style="font-size: small;">%s</span></p>'% (items_t[r[1]])
        HTMLResults.value += '<p style="text-align: center;"><span style="font-size: x-small;">Relativity score - %8.2f</span></p>'% (r[0])
        HTMLResults.value += '<hr /'
    display(PageTitle, HTMLResults)

search.on_click(searchclick)



[]
[[6.840248528409701, 458], [6.574530133972581, 613], [6.498123939447739, 483], [6.295264210796351, 259], [5.861464458859018, 29], [5.483595881249367, 587], [5.453025679027212, 43], [5.453025679027212, 120], [4.828552165405897, 698], [4.2495686645702015, 50], [4.222054566613885, 440], [4.131180665608722, 436], [4.01914730223191, 145], [3.979838088116913, 443], [3.9702073329398635, 951], [3.922444880468102, 584], [3.885054444614228, 482], [3.4937349376084055, 536], [3.40613975157642, 435], [3.3779093521722734, 110], [3.3709247186787734, 697], [3.276087425536348, 272], [3.1926807321827133, 590], [3.1864403977034885, 210], [3.1864403977034885, 228], [3.1864403977034885, 109], [3.1864403977034885, 113], [3.1434318804278423, 810], [3.1253530235848337, 445], [2.9715407094651085, 19], [2.913131538490633, 193], [2.8029412430812117, 898], [2.6698219891141317, 41], [2.2908950731151663, 869]]
[[9.429731118134065, 244], [8.790686208742583, 862], [6.497202343673924, 246], [5.761995917678025, 695]